In [ ]:
"""
Created on Thu Jan 16 14:55 2023

Trying to do overview plots like Pierre

Author: @claraburgard
"""

In [1]:
import xarray as xr
import numpy as np

import matplotlib.pyplot as plt
import matplotlib as mpl
import cmocean

import cartopy.crs as ccrs
import cartopy.feature
from cartopy.util import add_cyclic_point
import seaborn as sns
import cartopy
import matplotlib.colors as colors
from matplotlib.colors import Normalize
from cartopy.feature import LAND

from basal_melt_NEMO.constants import *
import basal_melt_NEMO.figure_functions as figf
import basal_melt_NEMO.useful_functions as uf

In [2]:
sns.set_context('paper')

In [3]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


READ IN DATA

In [4]:
inputpath_closed='/data/cdelaver/n42tm21/'
inputpath_open='/data/cdelaver/n42openc/'
clara_path='/data/cburgard/CASIMIR_SIMU/interim/CDO_PROCESSED/'

In [5]:
T_grid_vars_closed_mean10 = xr.open_dataset(inputpath_closed + 'n42tm21_00910101_01001231_1Y_grid_T.nc').mean('time_counter')
T_grid_vars_open_mean10 = xr.open_dataset(inputpath_open + 'n42openc_00910101_01001231_1Y_grid_T.nc').mean('time_counter')
T_grid_vars_open_mean10 = T_grid_vars_open_mean10.where(T_grid_vars_open_mean10['so'] > 0)

In [6]:
cellarea_closed = T_grid_vars_closed_mean10['area']
cellarea_open = T_grid_vars_open_mean10['area']

In [89]:
var_list = ['global_temp','wed_gyre','ross_gyre','ACC',
            'FRIS_melt','Ross_melt','LarsenC_melt','total_melt',
            'mar_sie_arc','sep_sie_arc','feb_sie_ant','sep_sie_ant',
            'salbot_WED','salbot_EWED','salbot_WROSS','salbot_EROSS',
            'theta_WED','theta_EWED','theta_WROSS','theta_EROSS']



In [91]:
var_obs_mean = xr.DataArray(data=np.array([15.0, 56.0, 20.0, 136.7, 
               155.4, 47.7, 20.7, 1500,
               15.3, 6.3, 3.3, 19.8, 
               34.9, np.nan, 35.0, np.nan, 
               np.nan, -1.95, -1.9, -1.7]), dims='var').assign_coords({'var': var_list})

In [92]:
var_obs_std = xr.DataArray(data=np.array([0, 8.0, 5.0, 7.8, 
              45.0, 34.0, 67.0, 237,
              1.0, 1.0, 0.5, 0.5, 
              0.0, np.nan, 0.0, np.nan,
              np.nan, 0.2, 0.4, 0.4]), dims='var').assign_coords({'var': var_list})

In [10]:
var_to_plot = xr.Dataset()

### GLOBAL SST
temp_open = xr.open_dataset(clara_path+'n42openc/n42openc_01-100_1Y_thetao_fldmean.nc').isel(deptht=0).squeeze()
temp_closed = xr.open_dataset(clara_path+'n42tm21/n42tm21_01-100_1Y_thetao_fldmean.nc').isel(deptht=0).squeeze()
var_to_plot['global_temp'] = xr.concat([temp_open['thetao'].assign_coords({'cav':'open'}),temp_closed['thetao'].assign_coords({'cav':'closed'})], dim='cav')

### ACC
u_open = xr.open_dataset(clara_path+'n42openc/n42openc_0-100_1Y_uo_vertsum.nc').squeeze()
u_closed = xr.open_dataset(clara_path+'n42tm21/n42tm21_0-100_1Y_uo_vertsum.nc').squeeze()
var_to_plot['ACC'] = xr.concat([u_open['uocetr_eff'].sel(x=220,y=range(79,107)).sum('y').assign_coords({'cav':'open'}),
                                u_closed['uocetr_eff'].sel(x=220,y=range(79,107)).sum('y').assign_coords({'cav':'closed'})], dim='cav')
var_to_plot['ACC'] = var_to_plot['ACC']/10**6

In [11]:
### GYRES
uocetr_Wed_open = u_open['uocetr_eff'].where(uf.in_range(u_open.nav_lat,[-66.50,-60.40]) & uf.in_range(u_open.nav_lon,[-31.25,37.50]), drop=True)
uocetr_Wed_closed = u_closed['uocetr_eff'].where(uf.in_range(u_closed.nav_lat,[-66.50,-60.40]) & uf.in_range(u_closed.nav_lon,[-31.25,37.50]), drop=True)

uocetr_Ross_open = u_open['uocetr_eff'].where(uf.in_range(u_open.nav_lat,[-72.650,-61.600]) & ((u_open.nav_lon <= -135.75) | (u_open.nav_lon >= 360-168.500)), drop=True)
uocetr_Ross_closed = u_closed['uocetr_eff'].where(uf.in_range(u_closed.nav_lat,[-72.650,-61.600]) & ((u_closed.nav_lon <= -135.75) | (u_closed.nav_lon >= 360-168.500)), drop=True)


In [12]:
var_to_plot['ross_gyre'] = xr.concat([uocetr_Ross_open.cumsum('y').max(['y','x']).assign_coords({'cav':'open'}), uocetr_Ross_closed.cumsum('y').max(['y','x']).assign_coords({'cav':'closed'})], dim='cav')/10**6
var_to_plot['wed_gyre'] = xr.concat([uocetr_Wed_open.cumsum('y').max(['y','x']).assign_coords({'cav':'open'}), uocetr_Wed_closed.cumsum('y').max(['y','x']).assign_coords({'cav':'closed'})], dim='cav')/10**6


In [13]:
### Iceshelf melt
#T_grid_vars_closed = xr.open_dataset(clara_path + 'n42tm21_00910101_01001231_1Y_grid_T.nc')
T_grid_vars_open = xr.open_dataset(clara_path + 'n42openc/n42openc_0-100_1Y_grid_isfvars.nc')
#T_grid_vars_open = T_grid_vars_open.where(T_grid_vars_open['so'] > 0)

lon = T_grid_vars_open.nav_lon
lat = T_grid_vars_open.nav_lat

mask_FRIS = (lon <= -30) & (lon >= -80.) & (lat >= -84.) & (lat <= -74.)
mask_Ross = ((lon <= -120) | (lon >= 150)) & (lat >= -87.) & (lat <= -65.)
mask_LarsenC = (lon <= -58) & (lon >= -66) & (lat >= -71.) & (lat <= -64.)

In [14]:
melt_all = (T_grid_vars_open['iceshelf_cav'] * T_grid_vars_open_mean10['area'].max('deptht') * yearinsec *10**(-12))
FRIS_melt= melt_all.where(mask_FRIS).sum(['x','y'])
Ross_melt = melt_all.where(mask_Ross).sum(['x','y'])
LarsenC_melt = melt_all.where(mask_LarsenC).sum(['x','y'])

var_to_plot['FRIS_melt'] = xr.concat([FRIS_melt.assign_coords({'cav':'open'}), FRIS_melt.assign_coords({'cav':'closed'}) * np.nan], dim='cav')
var_to_plot['Ross_melt'] = xr.concat([Ross_melt.assign_coords({'cav':'open'}), Ross_melt.assign_coords({'cav':'closed'}) * np.nan], dim='cav')
var_to_plot['LarsenC_melt'] = xr.concat([LarsenC_melt.assign_coords({'cav':'open'}), LarsenC_melt.assign_coords({'cav':'closed'}) * np.nan], dim='cav')

var_to_plot['total_melt'] = var_to_plot['FRIS_melt'] + var_to_plot['Ross_melt'] + var_to_plot['LarsenC_melt'] + (T_grid_vars_open['iceshelf'] * T_grid_vars_open_mean10['area'].isel(deptht=0) * yearinsec * 10**(-12)).sum(['x','y'])


In [39]:
### Sea ice
si_Ant_open = xr.open_dataset(clara_path+'n42openc/n42openc_01-100_1M_icemod_Ant_SIEmask.nc')
si_Ant_closed = xr.open_dataset(clara_path+'n42tm21/n42tm21_01-100_1M_icemod_Ant_SIEmask.nc').reindex_like(si_Ant_open)

In [45]:
si_Arc_open = xr.open_dataset(clara_path+'n42openc/n42openc_01-100_1M_icemod_Arc_SIEmask.nc')
si_Arc_closed = xr.open_dataset(clara_path+'n42tm21/n42tm21_01-100_1M_icemod_Arc_SIEmask.nc').reindex_like(si_Arc_open)

In [41]:
(si_Ant_closed['siconc']).where(si_Arc_closed['time_counter.month'] == 9, drop=True).isel(time_counter=89).plot()

In [22]:
plt.figure()
(si_Ant_open['siconc'] * cellarea_Ant_open).where(si_Arc_open['time_counter.month'] == 9, drop=True).isel(time_counter=89).plot()

In [31]:
cellarea_Ant_open.plot()

In [57]:
plt.figure()
(si_Ant_closed['siconc'] * cellarea_Ant_closed).where(si_Arc_closed['time_counter.month'] == 9, drop=True).isel(time_counter=89).plot()

In [30]:
cellarea_open.isel(deptht=0).plot()

In [133]:
si_Arc_closed['siconc'].where(si_Arc_closed['time_counter.month'] == 3, drop=True).isel(time_counter=89).plot()

In [60]:
cellarea_Ant_closed = cellarea_open.sel(x=si_Ant_closed.x, y=si_Ant_closed.y).isel(deptht=0)
cellarea_Arc_closed = cellarea_open.sel(x=si_Arc_closed.x, y=si_Arc_closed.y).isel(deptht=0)
cellarea_Ant_open = cellarea_open.sel(x=si_Ant_open.x, y=si_Ant_open.y).isel(deptht=0)
cellarea_Arc_open = cellarea_open.sel(x=si_Arc_open.x, y=si_Arc_open.y).isel(deptht=0)


In [64]:
sie_Ant_open = (si_Ant_open['siconc'] * cellarea_Ant_open).sum(['x','y']).drop('deptht')
sie_Ant_closed = (si_Ant_closed['siconc'] * cellarea_Ant_closed).sum(['x','y'])
sie_Arc_open = (si_Arc_open['siconc'] * cellarea_Arc_open).sum(['x','y']).drop('deptht')
sie_Arc_closed = (si_Arc_closed['siconc'] * cellarea_Arc_closed).sum(['x','y'])


In [65]:
var_to_plot['mar_sie_arc'] = xr.concat([sie_Arc_open.where(sie_Arc_open['time_counter.month'] == 3, drop=True).squeeze().assign_coords({'cav':'open'}),
                                        sie_Arc_closed.where(sie_Arc_closed['time_counter.month'] == 3, drop=True).squeeze().assign_coords({'cav':'closed'})], dim='cav').assign_coords({'time_counter': var_to_plot.time_counter})/10**12
var_to_plot['sep_sie_ant'] = xr.concat([sie_Ant_open.where(sie_Ant_open['time_counter.month'] == 9, drop=True).squeeze().assign_coords({'cav':'open'}),
                                        sie_Ant_closed.where(sie_Ant_closed['time_counter.month'] == 9, drop=True).squeeze().assign_coords({'cav':'closed'})], dim='cav').assign_coords({'time_counter': var_to_plot.time_counter})/10**12
var_to_plot['sep_sie_arc'] = xr.concat([sie_Arc_open.where(sie_Arc_open['time_counter.month'] == 9, drop=True).squeeze().assign_coords({'cav':'open'}),
                                        sie_Arc_closed.where(sie_Arc_closed['time_counter.month'] == 9, drop=True).squeeze().assign_coords({'cav':'closed'})], dim='cav').assign_coords({'time_counter': var_to_plot.time_counter})/10**12
var_to_plot['feb_sie_ant'] = xr.concat([sie_Ant_open.where(sie_Ant_open['time_counter.month'] == 2, drop=True).squeeze().assign_coords({'cav':'open'}),
                                        sie_Ant_closed.where(sie_Ant_closed['time_counter.month'] == 2, drop=True).squeeze().assign_coords({'cav':'closed'})], dim='cav').assign_coords({'time_counter': var_to_plot.time_counter})/10**12


In [69]:
T_grid_vars_closed

NameError: name 'T_grid_vars_closed' is not defined

In [76]:
T_grid_vars_open['so']

<xarray.DataArray 'so' (time_counter: 90, deptht: 75, y: 15, x: 8)>
[810000 values with dtype=float32]
Coordinates:
  * time_counter  (time_counter) object 0001-07-02 12:00:00 ... 0090-07-02 12...
    nav_lon       (y, x) float32 ...
    nav_lat       (y, x) float32 ...
  * deptht        (deptht) float32 0.5058 1.556 2.668 ... 5.698e+03 5.902e+03
Dimensions without coordinates: y, x
Attributes:
    standard_name:       sea_water_practical_salinity
    long_name:           sea_water_salinity
    units:               1e-3
    cell_measures:       area: cell_area
    online_operation:    average
    interval_operation:  1 yr
    interval_write:      1 yr
    cell_methods:        time: mean

In [101]:
### Bottom properties

for reg in ['AMU','WROSS','EROSS','WED','EWED']:
    T_grid_vars_closed = xr.open_dataset(clara_path + 'n42tm21/n42tm21_01-100_1Y_T_'+reg+'.nc')
    T_grid_vars_open = xr.open_dataset(clara_path + 'n42openc/n42openc_01-100_1Y_T_'+reg+'.nc')
    
    mask_cells = T_grid_vars_open['so'] > 0
    bottom_open = mask_cells.sum('deptht') - 1
    
    mask_cells = T_grid_vars_closed['so'] > 0
    bottom_closed = mask_cells.sum('deptht') - 1
    
    var_to_plot['theta_'+reg] = xr.concat([T_grid_vars_closed['thetao'].isel(deptht=bottom_closed).mean(['x','y']).assign_coords({'cav': 'closed'}),
                                           T_grid_vars_open['thetao'].isel(deptht=bottom_open).mean(['x','y']).assign_coords({'cav': 'open'})], dim='cav')
    var_to_plot['salbot_'+reg] = xr.concat([T_grid_vars_closed['so'].isel(deptht=bottom_closed).mean(['x','y']).assign_coords({'cav': 'closed'}),
                                           T_grid_vars_open['so'].isel(deptht=bottom_open).mean(['x','y']).assign_coords({'cav': 'open'})], dim='cav')

In [102]:
f = plt.figure()
f.set_size_inches(8.25*5, 8.25*5)

ax={}

leg_hdl = []

i = 0


for vv in var_list:
    
    ax[i] = f.add_subplot(5,5,i+1)

    if vv in list(var_to_plot.keys()):
        ax[i].plot(var_to_plot[vv].sel(cav='open'), color='deepskyblue')
        ax[i].plot(var_to_plot[vv].sel(cav='closed'), color='royalblue')
    
    ax[i].axhline(y=var_obs_mean.sel(var=vv), color='black', linewidth=2)
    ax[i].fill_between(x=np.arange(0,100),y1=var_obs_mean.sel(var=vv)-var_obs_std.sel(var=vv), y2=var_obs_mean.sel(var=vv)+var_obs_std.sel(var=vv), color='grey',alpha=0.2)

    ax[i].set_title(vv)

    i = i+1
#f.legend()
#f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()


In [100]:
T_grid_vars_closed['thetao'].isel(deptht=bottom_closed).isel(time_counter=0).plot()

In [ ]:
uocetr_Ross_open = u_open['uocetr_eff'].where(uf.in_range(u_open.nav_lat,[-72.650,-61.600]) & ((u_open.nav_lon <= -135.75) | (u_open.nav_lon >= 360-168.500)), drop=True)


In [42]:
uocetr_Ross_open.nav_lat.max()

<xarray.DataArray 'nav_lat' ()>
array(-61.72504807)

CHECK THAT I AM LOOKING AT THE RIGHT REGIONS FOR THE GYRES

In [108]:
ref = uocetr_Ross_open.isel(time_counter=0)

lon = ref.nav_lon
lat = ref.nav_lat

proj = ccrs.SouthPolarStereo(central_longitude=0)
wrap_ref, wrap_lon = ref, lon #add_cyclic_point(ref.values,coord=lon,axis=1)

theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpl.path.Path(verts * radius + center)   

ax1 = plt.subplot(1, 1, 1, projection=proj)
abso0 = ax1.pcolormesh(wrap_lon,lat,wrap_ref,transform=ccrs.PlateCarree(),rasterized=True)
ax1.coastlines(resolution='110m', linewidth=0.5)
ax1.set_extent([-180, 180, -90, -50], crs=ccrs.PlateCarree())
ax1.set_boundary(circle, transform=ax1.transAxes)

In [61]:
u_open['uocetr_eff'].sel(x=range(220,221),y=range(79,107)).isel(time_counter=0)

<xarray.DataArray 'uocetr_eff' (y: 28, x: 1)>
[28 values with dtype=float32]
Coordinates:
    time_counter  object 0001-07-02 12:00:00
    nav_lon       (y, x) float32 -66.0 -66.0 -66.0 -66.0 ... -66.0 -66.0 -66.0
    nav_lat       (y, x) float32 -67.3 -66.91 -66.51 ... -55.49 -54.92 -54.34
Dimensions without coordinates: y, x
Attributes:
    standard_name:       ocean_volume_x_transport
    long_name:           Effective ocean transport along i-axis
    units:               m3/s
    cell_measures:       area: cell_area
    online_operation:    average
    interval_operation:  2700 s
    interval_write:      1 yr
    cell_methods:        time: mean (interval: 2700 s)